In [24]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

from datetime import datetime, timedelta
import time

import os

import warnings
warnings.filterwarnings('ignore')

In [86]:
#A function get url, driver ,the information on reservation time and df and returns an updated df with the 100 additional hotels.
Unsuccessful_list  = [] # A list that saves the names of the hotels that have a problem in order to optimize and not try to extract information again
def get_hotels(url , driver ,booking_df,Snapshot_date,checking_date_type,checkout_date_type):
    driver.get(url)
    time.sleep(1)
    hotel_name_list =[]
    page_number=1
    index = 1
    while(len(hotel_name_list) <100 and page_number<=5): #take 100 hotel 
        time.sleep(2)
        text_box = driver.find_elements(by= By.CLASS_NAME , value='b978843432')
        for t_box in text_box:# run on full page
            if (len(hotel_name_list) >= 100):
                break
            try:
                hotel = {'Snapshot' : None ,'Index':None,'Hotel Name':None,'TTT':None,'LOS':None ,"Grade":None,"Num of Reviews":None,"Curr Price":None,"Original Price":None,"Distance from center":None,"Type of room":None,"Location grade":None ,"Extra Data":None}
                name = t_box.find_element(by= By.CLASS_NAME , value="e13098a59f").text
                name = name.split('\n')[0]
                if(name in hotel_name_list):#Avoid hotels that are advertised several times
                    continue   
                if (name in Unsuccessful_list):#Avoid hotels that we cannot extract
                    continue
                hotel['Hotel Name'] = (name)
                hotel['Snapshot'] = Snapshot_date
                delta_TTT =  checking_date_type - Snapshot_date
                hotel['TTT'] = delta_TTT.days
                delta_LOS = checkout_date_type - checking_date_type
                hotel['LOS'] = delta_LOS.days
                score_detail = t_box.find_element(by =By.CSS_SELECTOR,value ='[data-testid="review-score"]').text
                score_detail_list = score_detail.split('\n')
                hotel["Grade"] = (score_detail_list[0])
                hotel["Num of Reviews"] = (score_detail_list[2])
                hotel["Curr Price"] = (t_box.find_element(by =By.CSS_SELECTOR,value ='[data-testid="price-and-discounted-price"]').text)
                hotel["Index"] = index
                index = index +1
                hotel["Extra Data"] = t_box.find_element(by= By.CLASS_NAME , value="d22a7c133b").text
                hotel["Distance from center"] =  t_box.find_element(by =By.CSS_SELECTOR,value ='[data-testid="distance"]').text
                hotel["Type of room"] = t_box.find_element(by= By.CLASS_NAME , value="df597226dd").text
                
                try:
                    hotel["Location grade"] = t_box.find_element(by= By.CLASS_NAME , value="f9afbb0024").text 
                except:
                    hotel["Location grade"] = None
            
                try:
                    hotel["Original Price"] = t_box.find_element(by =By.XPATH,value ='.//*[@class="c5888af24f e729ed5ab6"]').text     
                except:
                    hotel["Original Price"] = hotel["Curr Price"]
                
                booking_df = booking_df.append(hotel, ignore_index=True)#add the new hotel
                hotel_name_list.append(name) #add the new hotel name
            except:
                print("error " + name)
                Unsuccessful_list.append(name)
        # change page
        if( len(hotel_name_list)< 100): 
            button=driver.find_element(by =By.CSS_SELECTOR,value =f'[aria-label=" {page_number+1}"]')
            time.sleep(1.5)
            button.click() 
            page_number = page_number+1
    
    
    return booking_df
    

In [77]:
#Allows us to run on different computers without a problem

path ={'roy' :'C:\Windows\chromdriver.exe' , 'yarin': 0 , 'yair':0 }
driver = webdriver.Chrome(path['roy'])

In [78]:
# Get today's date
now = datetime.now()
Snapshot_date =now
# Get end date 35 days from now
end_date = now + timedelta(days=35)

# Create a list of all dates between today and end date
date_list = []
while now <= end_date:
    date_list.append(now.strftime('%Y-%m-%d'))
    now += timedelta(days=1)


#Creates a list whose every member is a list of checking and checkout time

combinations_list =[]
for i in range(1,len(date_list)-5):
    for j in range(1,6):
        combinations = [date_list[i],date_list[i+j]]
        combinations_list.append(combinations)
        
print(combinations_list , len(combinations_list))        
    

[['2023-02-05', '2023-02-06'], ['2023-02-05', '2023-02-07'], ['2023-02-05', '2023-02-08'], ['2023-02-05', '2023-02-09'], ['2023-02-05', '2023-02-10'], ['2023-02-06', '2023-02-07'], ['2023-02-06', '2023-02-08'], ['2023-02-06', '2023-02-09'], ['2023-02-06', '2023-02-10'], ['2023-02-06', '2023-02-11'], ['2023-02-07', '2023-02-08'], ['2023-02-07', '2023-02-09'], ['2023-02-07', '2023-02-10'], ['2023-02-07', '2023-02-11'], ['2023-02-07', '2023-02-12'], ['2023-02-08', '2023-02-09'], ['2023-02-08', '2023-02-10'], ['2023-02-08', '2023-02-11'], ['2023-02-08', '2023-02-12'], ['2023-02-08', '2023-02-13'], ['2023-02-09', '2023-02-10'], ['2023-02-09', '2023-02-11'], ['2023-02-09', '2023-02-12'], ['2023-02-09', '2023-02-13'], ['2023-02-09', '2023-02-14'], ['2023-02-10', '2023-02-11'], ['2023-02-10', '2023-02-12'], ['2023-02-10', '2023-02-13'], ['2023-02-10', '2023-02-14'], ['2023-02-10', '2023-02-15'], ['2023-02-11', '2023-02-12'], ['2023-02-11', '2023-02-13'], ['2023-02-11', '2023-02-14'], ['2023-02

In [87]:
city = "new york"
numOfAdults=2
numOfChild=0
numOfRooms=1
booking_df = pd.DataFrame(columns=['Snapshot','Index','Hotel Name','TTT' ,'LOS' ,"Grade","Num of Reviews","Curr Price","Original Price","Distance from center","Type of room","Location grade", "Extra Data"])  
Unsuccessful_list = []
for comb in combinations_list:#run on the combinations list
    #get the information on reservation time 
    checking_date_type = datetime.strptime(comb[0], '%Y-%m-%d')
    checkout_date_type = datetime.strptime(comb[1], '%Y-%m-%d')
    checking_date = comb[0].split('-')
    checkout_date = comb[1].split('-')
    checking_year = checking_date[0]
    checking_month = checking_date[1]
    checking_monthday =checking_date[2]
    checkout_year= checkout_date[0]
    checkout_month= checkout_date[1]
    checkout_monthday= checkout_date[2]
    url = f"https://www.booking.com/searchresults.html?ss={city}&checkin_year={checking_year}&checkin_month={checking_month}&checkin_monthday={checking_monthday}&checkout_year={checkout_year}&checkout_month={checkout_month}&checkout_monthday={checkout_monthday}&group_adults={numOfAdults}&group_children={numOfChild}&no_rooms={numOfRooms}"
    booking_df = get_hotels(url, driver , booking_df, Snapshot_date,checking_date_type,checkout_date_type)
    

In [88]:
booking_df.head(6500)

,Snapshot,Index,Hotel Name,TTT,LOS,Grade,Num of Reviews,Curr Price,Original Price,Distance from center,Type of room,Location grade,Extra Data
0,2023-02-04 19:27:16.386035,1,Pestana CR7 Times Square,0,1,8.0,"1,301 reviews",₪ 363,₪ 605,1.6 km from center,Comfort Room - Disability Access,None,Comfort Room - Disability Access\n1 queen bed\...
1,2023-02-04 19:27:16.386035,2,Riu Plaza Manhattan Times Square,0,1,8.6,"6,055 reviews",₪ 474,₪ 658,1 km from center,Deluxe King,Location 9.5,Deluxe King\n1 king bed
2,2023-02-04 19:27:16.386035,3,Pod Times Square,0,1,8.1,"12,566 reviews",₪ 304,₪ 304,1.4 km from center,Standard Bunk Bed Room,Location 9.3,Standard Bunk Bed Room\n1 bunk bed\nOnly 5 roo...
3,2023-02-04 19:27:16.386035,4,Hyatt Place New York City/Times Square,0,1,7.6,"14,671 reviews",₪ 427,₪ 427,1.7 km from center,King Room,None,King Room\n1 king bed\nBreakfast included
4,2023-02-04 19:27:16.386035,5,Courtyard by Marriott New York Manhattan/Centr...,0,1,8.2,"3,232 reviews",₪ 612,₪ 612,450 m from center,King or Queen Room with City View,Location 9.4,King or Queen Room with City View\n1 queen bed...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2023-02-04 19:27:16.386035,96,SpringHill Suites by Marriott New York Midtown...,0,1,8.1,49 reviews,₪ 544,₪ 544,3.1 km from center,"Studio, 1 King",Location 9.3,"Studio, 1 King\n1 king bed\nBreakfast included..."
96,2023-02-04 19:27:16.386035,97,Roxy Hotel New York,0,1,8.4,953 reviews,₪ 680,₪ 680,5.8 km from center,Standard Room,Location 9.3,Standard Room\n1 full bed
97,2023-02-04 19:27:16.386035,98,The Muse New York,0,1,8.1,838 reviews,₪ 851,₪ 851,1.1 km from center,Accessible Deluxe King,None,Accessible Deluxe King\n1 king bed
98,2023-02-04 19:27:16.386035,99,The Shelburne Sonesta New York,0,1,7.9,"1,146 reviews",₪ 415,₪ 415,2.2 km from center,Queen Room,None,Queen Room\n1 queen bed


In [13]:
booking_df.shape

(15000, 14)

In [14]:
booking_df.groupby(['TTT','LOS']).count()

Snapshot  Index  Hotel Name  Num of stars  Grade  Num of Reviews  \
TTT LOS                                                                     
0   1         100    100         100             0    100             100   
    2         100    100         100             0    100             100   
    3         100    100         100             0    100             100   
    4         100    100         100             0    100             100   
    5         100    100         100             0    100             100   
...           ...    ...         ...           ...    ...             ...   
29  1         100    100         100             0    100             100   
    2         100    100         100             0    100             100   
    3         100    100         100             0    100             100   
    4         100    100         100             0    100             100   
    5         100    100         100             0    100             100   

         Curr Price  Original Price  Percentage of discount (Boolean)  \
TTT LOS                                                                 
0   1           100               0                                 0   
    2           100               0                                 0   
    3           100               0                                 0   
    4           100               0                                 0   
    5           100               0                                 0   
...             ...             ...                               ...   
29  1           100               0                                 0   
    2           100               0                                 0   
    3           100               0                                 0   
    4           100               0                                 0   
    5           100               0                                 0   

         Distance from center  Type of room  Location grade  
TTT LOS                                                      
0   1                       0             0               0  
    2                       0             0               0  
    3                       0             0               0  
    4                       0             0               0  
    5                       0             0               0  
...                       ...           ...             ...  
29  1                       0             0               0  
    2                       0             0               0  
    3                       0             0               0  
    4                       0             0               0  
    5                       0             0               0  

[150 rows x 12 columns]

In [89]:
driver.quit()

save the df in csv

In [32]:
file_name = "Booking_hotels.csv"
if os.path.exists(file_name):#if the "Booking_hotels.csv" exists
    print("exists")
    booking_df.to_csv(file_name,mode = 'a' , index= False ,header=False)#append without the header
else:
    print("new")
    booking_df.to_csv(file_name,mode = 'a' , index= False ,header=True)#append with the header

new
